# Case study: Lorenz attractor

This notebook contains a full TDA pipeline to analyse the transitions of the Lorenz system to a chaotic regime from the stable one and viceversa.

The notebook starts by importing the *topological learning library*.

In [ ]:
# Import TL library
import topological_learning as tl
import topological_learning.datasets as ds
import topological_learning.preprocessing as prep
import topological_learning.diagram as diag
import topological_learning.homology as hl
import numpy as np
import pandas as pd
import datetime as dt
import sklearn as sk
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
import topological_learning.manifold as ma

test = tl.manifold.StatefulMDS()

# Setting up the Lorenz attractor simulation

In the next block we set up all the parameters of the Lorenz system and we define also the instants at which the regime (stable VS chaotic) changes.

In [ ]:
max_duration = 20000
transition_list = [0.037500000000000006, 0.047, 0.0625, 0.077, 0.10350000000000001, 0.1275, 0.1585, 0.1615, 
                  0.1685, 0.2215, 0.23, 0.23700000000000002, 0.2515, 0.2715, 0.3, 0.30750000000000005, 0.3385, 
                  0.36350000000000005, 0.388, 0.40449999999999997, 0.41950000000000004, 0.4225, 0.4515, 
                  0.46950000000000003, 0.506, 0.5405, 0.542, 0.5825, 0.5825, 0.6144999999999999, 0.635, 0.66, 
                  0.677, 0.6845, 0.685, 0.729, 0.749, 0.769, 0.777, 0.8075, 0.8310000000000001, 0.833, 0.864, 
                  0.8674999999999999, 0.882, 0.903, 0.9395, 0.94, 0.974, 0.99]

# Method to generate a solution of the Lorenz system
lorenzAttractor = ds.LorenzDataset(initial_conditions=(1, -10, 10), sigma=10., beta=8./3., rho_min=5, rho_max=15,
                     transition_list=transition_list, number_transitions=25, transition_duration=100,
                     time_step=0.01, max_duration=max_duration, mean_noise=0., std_deviation_noise=0.0)
lorenzAttractor.run()

In [ ]:
# Plotting the trajectories of the Lorenz system
trace = go.Scatter3d(x=lorenzAttractor.x_, y=lorenzAttractor.y_, z=lorenzAttractor.z_, mode='markers',
                     marker=dict(size=4, color=list(range(max_duration)),
                     colorscale='Viridis', opacity=0.8))

data = [trace]
layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))

figure = go.Figure(data=data, layout=layout)
iplot(figure)

In [ ]:
# Set up the dataframe of the z-projection of the simulated solution
zDataFrame = pd.DataFrame(lorenzAttractor.z_)
index = pd.to_datetime(zDataFrame.index, utc=True, unit='h')
zDataFrame.index = index

# Plotting the Lorenz system
fig = plt.figure(figsize=(16,6))
plt.plot(zDataFrame)
plt.title('Trajectory of the Lorenz solution, projected along the z-axis')
plt.show()

# Resampling the time series

It is important to find the correct time scale at which key signals take place. Here we propose one possible resampling period: *10h*. Recall that the unit time is *1h*. The resampler method is used to perform the resampling.

In [ ]:
samplingPeriod = '10h'
periodicSampler = prep.Resampler(sampling_type='periodic', sampling_period=samplingPeriod, remove_weekends=False)

In [ ]:
periodicSampler.fit(zDataFrame)
zDataFrameSampled = periodicSampler.transform(zDataFrame)

fig = plt.figure(figsize=(16,6))
plt.plot(zDataFrameSampled)
plt.title('Trajectory of the Lorenz solution, projected along the z-axis and resampled every 10h')
plt.show()

# Takens Embedding

In order to obtain meaningful topological features from a time series, we use a delayed-time embedding technique, invented by F. Takens in the late sixties.
The idea is simple: given a time series X(t), one can extract a sequence of vectors of the form X_i := [(X(t_i)), X(t_i + 2 tau), ..., X(t_i + M tau)].
The difference between t_i and t_i-1 is called *stride*; the numbers M and tau are optimized authomatically in this example (they can be set by the user if needed).
    
The *outer window* allows us to apply Takens embedding locally on a certain interval rather than over the whole time series. The result of this procedure is therefore a time series of point clouds with possibly interesting topologies.


In [ ]:
outerWindowDuration = 100
outerWindowStride = 20
embedder = prep.TakensEmbedder(outer_window_duration=outerWindowDuration, outer_window_stride=outerWindowStride, 
                              embedding_parameters_type='search', embedding_dimension=10, embedding_time_delay=3, 
                              n_jobs=-1)

In [ ]:
embedder.fit(zDataFrameSampled)
zEmbedded = embedder.transform(zDataFrameSampled)
print('Optimal embedding time delay based on mutual information: ', embedder.embedding_time_delay)
print('Optimal embedding dimension based on false nearest neighbors: ', embedder.embedding_dimension)

In [ ]:
windowNumber = 3

In [ ]:
# Plotting Takens embedding of the first outer window
window = zEmbedded[windowNumber]
trace = go.Scatter3d(x=window[:, 0], y=window[:, 1], z=window[:, 2], mode='markers',
                     marker=dict(size=4, color=list(range(window.shape[0])),
                     colorscale='Viridis', opacity=0.8))

data = [trace]
layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))

figure = go.Figure(data=data, layout=layout)
iplot(figure)

In [ ]:
# Plotting the time series associated to the above point cloud. 
# Notice the two periodicities corresponding to the loops in the embedding
fig = plt.figure(figsize=(16,6))
plt.plot(zDataFrameSampled[windowNumber*outerWindowStride:windowNumber*outerWindowStride + outerWindowDuration])
plt.title('The Lorenz solution over one outer window ')
plt.show()

# Persistence diagram
The topological information of the correlation metric is synthesised in the persistent diagram. The horizonral axis corresponds to the moment in which an homological generator is born, while the vertical axis corresponds to the moments in which an homological generator dies.
The generators of the homology groups (at given rank) are colored differently.

In [ ]:
homologyDimensions = [0, 1 ,2]
persistenceDiagram = hl.VietorisRipsPersistence(metric='euclidean', max_edge_length=100, 
                                            homology_dimensions=homologyDimensions, n_jobs=-1)
persistenceDiagram.fit(zEmbedded)
zDiagrams = persistenceDiagram.transform(zEmbedded)

In [ ]:
fig = plt.figure(figsize=(10,6))
diagram = { dimension: zDiagrams[dimension][windowNumber] for dimension in homologyDimensions }

for dimension in homologyDimensions:
    plt.plot(diagram[dimension][:,0], diagram[dimension][:,1], 'o',label='homology '+ str(dimension))
plt.title('Persistent Diagram')
plt.legend(loc='lower right')
plt.plot([0, 22], [0, 22], color='k')
plt.show()

# Distances among diagrams

In this section we show how to compute distances among persistent diagrams. There are many notions of distances: here we use the *bottleneck distance* as an example.

We stress that the *i-th* row of this matrix is the distance of the *i-th* diagram from all the others.

In [ ]:
# Computing the bottleneck distance of the time series of persistent diagrams
diagramDistance = diag.DiagramDistance(metric='bottleneck', metric_params={'order': np.inf}, n_jobs=-1)
diagramDistance.fit(zDiagrams)
zDistanceMatrix_BN = diagramDistance.transform(zDiagrams)

In [ ]:
figure = plt.figure(figsize=(10,10))
plt.imshow(zDistanceMatrix_BN)
plt.colorbar()
plt.title('Bottleneck distance matrix')
plt.show()

In [ ]:
# We change metric: we compute the landscape distance among the diagrams 
diagramDistance = diag.DiagramDistance(metric='landscape', metric_params={'order': 2, 'n_layers':1, 'n_samples':1000}, 
                                       n_jobs=-1)
diagramDistance.fit(zDiagrams)
zDistanceMatrix_L = diagramDistance.transform(zDiagrams)

In [ ]:
# Plot of the landascape L2 distance between persistent diagrams 
figure = plt.figure(figsize=(10,10))
plt.imshow(zDistanceMatrix_L)
plt.colorbar()
plt.title('Landscape L2 distance matrix')
plt.show()

# Adding gaussian noise

It is claimed in the literature that TDA is robust with respect to noise. Let us then add noise to the system and redo the previous analysis and see what changes! 

In [ ]:
lorenzAttractor.stdDeviationNoise = 0.3
lorenzAttractor.run()

In [ ]:
zDataFrame_noise = pd.DataFrame(lorenzAttractor.z_)
index = pd.to_datetime(zDataFrame_noise.index, utc=True, unit='h')
zDataFrame_noise.index = index

fig = plt.figure(figsize=(16,6))
plt.plot(zDataFrame_noise)
plt.show()

# Scikit-learn *Pipeline*
One of the advantages of our topological learning library is the compatibility with scikit-learn. It is possible to set up a full pipeline such as the one above in a few lines. We will show how in the next section.



In [ ]:
# Steps of the Pipeline
steps = [
    ('sampling', prep.Resampler(sampling_type='periodic', sampling_period=samplingPeriod, remove_weekends=False)),
    ('embedding', prep.TakensEmbedder(outer_window_duration=outerWindowDuration, outer_window_stride=outerWindowStride, 
                              embedding_parameters_type='search', embedding_dimension=10, embedding_time_delay=3, 
                              n_jobs=-1)),
    ('diagrams', hl.VietorisRipsPersistence(metric='euclidean', max_edge_length=100, 
                                        homology_dimensions=homologyDimensions, n_jobs=-1))
]

# Sklearn Pipeline
pipeline = Pipeline(steps)

In [ ]:
# Running the pipeline
pipeline.fit(zDataFrame_noise)
zDiagrams_noise = pipeline.transform(zDataFrame_noise)

In [ ]:
# Plotting the final persistent diagram of one outer window
fig = plt.figure(figsize=(10,6))
diagram = { dimension: zDiagrams_noise[dimension][windowNumber] for dimension in homologyDimensions }

for dimension in homologyDimensions:
    plt.plot(diagram[dimension][:,0], diagram[dimension][:,1], 'o',label='homology '+ str(dimension))
plt.title('Persistent Diagram')
plt.legend(loc='lower right')
plt.plot([0, 25], [0, 25], color='k')
plt.show()

# Rescaling the diagram
Rescaling the diagram means normalizing the points such that their distance from the *empty diagram* is equal to one.
Once the diagram is rescaled, we can filter noise by removing all the points that are close to the diagonal.

In [ ]:
diagramScaler = diag.DiagramScaler()
diagramScaler.fit(zDiagrams_noise)
zDiagramsScaled_noise = diagramScaler.transform(zDiagrams_noise)

In [ ]:
fig = plt.figure(figsize=(10,6))
diagram = { dimension: zDiagramsScaled_noise[dimension][windowNumber] for dimension in homologyDimensions }

for dimension in homologyDimensions:
    plt.plot(diagram[dimension][:,0], diagram[dimension][:,1], 'o',label='homology '+ str(dimension))
plt.title('Rescaled Persistent Diagram')
plt.legend(loc='lower right')
plt.plot([0, 2], [0, 2], color='k')
plt.show()

# Filtering diagrams

Filtering noise from a diagram corresponds to eliminating all the homology generators whose lifespan is too short to be significant. It means equivalently that we are removing the points closer to the diagonal than a centrain threshold.

In [ ]:
# Perparing the filtering transformer
diagramFilter = diag.DiagramFilter(delta=0.05, homology_dimensions={1, 2})
diagramFilter.fit(zDiagramsScaled_noise)
zDiagramsFiltered_noise = diagramFilter.transform(zDiagramsScaled_noise)

In [ ]:
# Plot of the filtered diagram
fig = plt.figure(figsize=(10,6))
diagram = { dimension: zDiagramsFiltered_noise[dimension][windowNumber] for dimension in homologyDimensions }

for dimension in homologyDimensions:
    plt.plot(diagram[dimension][:,0], diagram[dimension][:,1], 'o',label='homology '+ str(dimension))
plt.title('Filtered Persistent Diagram')
plt.legend(loc='lower right')
plt.plot([0, 2], [0, 2], color='k')
plt.show()

In [ ]:
# Wrapping up all the steps inside a scikit-learn pipeline
steps = [
    ('sampling', prep.Resampler(sampling_type='periodic', sampling_period=samplingPeriod, remove_weekends=False)),
    ('embedding', prep.TakensEmbedder(outer_window_duration=outerWindowDuration, outer_window_stride=outerWindowStride, 
                              embedding_parameters_type='search', embedding_dimension=10, embedding_time_delay=10, 
                              n_jobs=-1)),
    ('diagrams', hl.VietorisRipsPersistence(metric='euclidean', max_edge_length=100, 
                                        homology_dimensions=homologyDimensions, n_jobs=-1)),
    ('diagrams_scaler', diag.DiagramScaler()),
    ('diagrams_filter', diag.DiagramFilter(delta=0.05))
]

pipeline_filter = Pipeline(steps)

In [ ]:
pipeline_filter.fit(zDataFrame_noise)
zDiagramsFilteredPipeline_noise = pipeline_filter.transform(zDataFrame_noise)

In [ ]:
# Plot the filtered diagram
fig = plt.figure(figsize=(10,6))
diagram = { dimension: zDiagramsFilteredPipeline_noise[dimension][windowNumber] for dimension in homologyDimensions }

for dimension in homologyDimensions:
    plt.plot(diagram[dimension][:,0], diagram[dimension][:,1], 'o',label='homology '+ str(dimension))
plt.title('Filtered Persistent Diagram')
plt.legend(loc='lower right')
plt.plot([0, 2], [0, 2], color='k')
plt.show()

# New distance in the embedding space: permutations and graphs

We propose here a new way to compute distances between points in the embedding space. Instead of considering the Euclidean distance in the Takens space, we propose to build a kNN-graph and then use the geodesic distance on such graph.

In [ ]:
nnEmbedder = prep.NearestNeighborGraphEmbedder(n_neighbors=3, n_jobs=1)
nnEmbedder.fit(zEmbedded)
zNNGraph = nnEmbedder.transform(zEmbedded[0].reshape((1, zEmbedded.shape[1], -1)))

# Geodesic distance on graphs
Given the graph embedding, the natural notion of distance between vertices corresponds to the lengths of the shortest path connecting two vertices. This is also known as *graph geodesic distance*. 

In [ ]:
geodesicDistance = prep.GeodesicDistance(n_jobs=1)
geodesicDistance.fit(zNNGraph)
zGeoNN = geodesicDistance.transform(zNNGraph)

In [ ]:
# Plotting the geodesic distance
figure = plt.figure(figsize=(10,10))
plt.imshow(zGeoNN[0])
plt.colorbar()
plt.title('Plot of the geodesic distance of the first outer window kNN graph')
plt.show()

# New embedding: permutations and graphs

We propose here a new way to transform the time series outer windows into graphs rather than point clouds. The idea is to create a graph whose vertices are permutation patterns and the edges exists between two vertices if the corresponding patterns happened one after the other.

In [ ]:
permutationEmbedder = prep.PermutationEmbedder(n_jobs=-1)
permutationEmbedder.fit(zEmbedded)
zPermEmbedded = permutationEmbedder.transform(zEmbedded)

In [ ]:
# Permutation embedding of the first outer window
window = zPermEmbedded[windowNumber]
trace = go.Scatter3d(x=window[:, 0], y=window[:, 1], z=window[:, 2], mode='markers',
                     marker=dict(size=4, color=list(range(window.shape[0])),
                     colorscale='Viridis', opacity=0.8))

data = [trace]
layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))

figure = go.Figure(data=data, layout=layout)
iplot(figure)

# Synthetic features extracted from topology

One can use persistent diagrams as he pleases. Here we show one way of extracting summary information from the time-series of diagrams: the *permutation entropy*. The entropy is computed by estimating a probability based on teh counting how many permutation patterns appear along the time series within the outer window.

In [ ]:
# Computing permutation entropy
permutationEntropy = prep.PermutationEntropy(n_jobs=1)
permutationEntropy.fit(zPermEmbedded)
zPermEntropy = permutationEntropy.transform(zPermEmbedded)

In [ ]:
# Plot the sampled Lorenz solution, projected along the z-axis
fig = plt.figure(figsize=(16,6))
plt.plot(zDataFrameSampled)
plt.title('Resampled solution of the Lorenz attractor projected along the z-axis')
plt.show()

In [ ]:
# Plot of the permutation entropy time-series
fig = plt.figure(figsize=(16,6))
plt.plot(zPermEntropy)
plt.title('Permutation entropy')
plt.show()

# Computing graph distances between the time series of graphs
The procedure we have developed associateds to each outer window a graph. This graph is a subgraph of the full permutation graph. The distance among subgraphs is then defined via the geodesic distance.

In [ ]:
graphEmbedder = prep.UniqueGraphEmbedder(n_jobs=1)
graphEmbedder.fit(zPermEmbedded)
zPermGraph = graphEmbedder.transform(zPermEmbedded)

In [ ]:
# Computing geodesic distance between subgraph inside the universal graph
geodesicDistance = prep.GeodesicDistance(n_jobs=1)
geodesicDistance.fit(zPermGraph)
zGeoPerm = geodesicDistance.transform(zPermGraph)

In [ ]:
# Plot of the geodesci distance between the time series of subgraphs
figure = plt.figure(figsize=(10,10))
plt.imshow(zGeoPerm[windowNumber])
plt.colorbar()
plt.title('Geodesic graph distance between subgraphs.')
plt.show()